In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.dont_write_bytecode = True
import json
import time
from datetime import datetime
import single_node_profiles_cpp as snp
import profiler
import end_to_end_profiles as e2e_profs
import numpy as np
from optimizer import BruteForceOptimizer, GreedyOptimizer
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
profs = snp.load_single_node_profiles()

In [4]:
profs.keys()

[u'tf-resnet-feats',
 u'tf-log-reg',
 u'tf-kernel-svm',
 u'res50',
 u'inception',
 u'res152',
 u'alexnet']

In [5]:
dag = profiler.get_logical_pipeline("pipeline_one")
with open(os.path.abspath("../results/e2e_profs_new_metrics/incep_1-logreg_1-ksvm_1-resnet_1-180312_012713.json")) as f:
    sample_run = json.load(f)
print(dag.reference_node)
scale_factors = profiler.get_node_scale_factors(sample_run, dag.reference_node)
node_configs = profiler.get_node_configs_from_experiment(sample_run)
def which_stage(model_name):
    if model_name == "tf-kernel-svm" or model_name == "tf-log-reg":
        return "latency_stage"
    else:
        return "thru_stage"
node_profs = {name : profiler.NodeProfile(name, profs[name], which_stage(name)) for name, _ in node_configs.items()}

inception


In [25]:
inception = profiler.NodeProfile("inception", profs["inception"], "thru_stage")
inception.profile

,p99_latency,num_cpus_per_replica,thru_stage_std_throughput_qps,mean_queue_size,cost,cloud,thru_stage_mean_throughput_qps,gpu_type,mean_batch_size,latency_stage_std_throughput_qps,fname,latency_stage_mean_throughput_qps
0,0.156590,1,0.123469,0.0,0.7005,aws,57.012517,k80,8.000000,0.224940,under_over-cpp-aws-results-k80-inception-batch...,54.919579
1,0.395980,1,6.941721,0.0,2.5945,aws,324.966044,v100,95.314286,8.154756,under_over-cpp-aws-results-v100-inception-batc...,257.266206
2,0.361722,1,0.296977,0.0,0.7005,aws,71.681687,k80,24.000000,0.496270,under_over-cpp-aws-results-k80-inception-batch...,68.428146
3,0.075928,1,1.538542,0.0,2.5945,aws,208.058480,v100,12.000000,0.829982,under_over-cpp-aws-results-v100-inception-batc...,181.028917
4,0.043805,1,0.198021,0.0,2.5945,aws,29.128397,v100,1.000000,0.103844,under_over-cpp-aws-results-v100-inception-batc...,28.940086
5,0.078155,1,0.029107,0.0,0.7005,aws,30.207227,k80,2.000000,0.054996,under_over-cpp-aws-results-k80-inception-batch...,29.779137
6,0.200831,1,2.693090,0.0,2.5945,aws,325.368670,v100,48.000000,3.586959,under_over-cpp-aws-results-v100-inception-batc...,260.643720
7,0.051343,1,0.896815,0.0,2.5945,aws,97.063449,v100,4.000000,0.754405,under_over-cpp-aws-results-v100-inception-batc...,93.811766
8,0.104188,1,0.046419,0.0,0.7005,aws,44.220629,k80,4.000000,0.102239,under_over-cpp-aws-results-k80-inception-batch...,42.726823
9,0.089602,1,0.025880,0.0,0.7005,aws,38.648904,k80,3.000000,0.060364,under_over-cpp-aws-results-k80-inception-batch...,37.749247


In [24]:
dag

In [30]:
opt = GreedyOptimizer(dag, scale_factors, node_profs)
cloud = "aws"
initial_config = {"tf-resnet-feats": profiler.NodeConfig(name="tf-resnet-feats",
                                                          num_cpus=1,
                                                          gpu_type="v100",
                                                          batch_size=1,
                                                          num_replicas=1,
                                                          cloud=cloud),
                  "inception": profiler.NodeConfig(name="inception",
                                                      num_cpus=1,
                                                      gpu_type="v100",
                                                      batch_size=1,
                                                      num_replicas=1,
                                                      cloud=cloud),
                  "tf-log-reg": profiler.NodeConfig(name="tf-log-reg",
                                                      num_cpus=1,
                                                      gpu_type="none",
                                                      batch_size=1,
                                                      num_replicas=1,
                                                      cloud=cloud),
                  "tf-kernel-svm": profiler.NodeConfig(name="tf-kernel-svm",
                                                      num_cpus=1,
                                                      gpu_type="none",
                                                      batch_size=1,
                                                      num_replicas=1,
                                                      cloud=cloud),
                 }
opt.select_optimal_config(cloud, 0.5, 50, initial_config)

0
Upgrading bottleneck node tf-resnet-feats to NodeConfig(tf-resnet-feats, 1, v100, 1, 2, aws)
0
Upgrading bottleneck node tf-resnet-feats to NodeConfig(tf-resnet-feats, 1, v100, 1, 3, aws)
0
Upgrading bottleneck node inception to NodeConfig(inception, 1, v100, 1, 2, aws)
0
Upgrading bottleneck node tf-resnet-feats to NodeConfig(tf-resnet-feats, 1, v100, 2.0, 3, aws)
0
Upgrading bottleneck node inception to NodeConfig(inception, 1, v100, 1, 3, aws)
0
Upgrading bottleneck node tf-resnet-feats to NodeConfig(tf-resnet-feats, 1, v100, 2.0, 4, aws)
0
Upgrading bottleneck node tf-kernel-svm to NodeConfig(tf-kernel-svm, 1, none, 1, 2, aws)
0
Upgrading bottleneck node tf-resnet-feats to NodeConfig(tf-resnet-feats, 1, v100, 2.0, 5, aws)
0
Upgrading bottleneck node inception to NodeConfig(inception, 1, v100, 1, 4, aws)
0
Upgrading bottleneck node tf-resnet-feats to NodeConfig(tf-resnet-feats, 1, v100, 2.0, 6, aws)
0
Upgrading bottleneck node inception to NodeConfig(inception, 1, v100, 1, 5, aws)

({'inception': NodeConfig(inception, 1, v100, 95.3142857143, 8, aws),
  'tf-kernel-svm': NodeConfig(tf-kernel-svm, 1, none, 8.0, 7, aws),
  'tf-log-reg': NodeConfig(tf-log-reg, 1, none, 2.0, 3, aws),
  'tf-resnet-feats': NodeConfig(tf-resnet-feats, 1, v100, 96.0, 11, aws)},
 {'cost': 49.960499999999996,
  'latency': 0.46773674,
  'throughput': 2599.7283542857144})

In [ ]:
import itertools
def brute_force_optimizer(dag, scale_factors, node_profs, cost_constraint, latency_constraint):
    """
        This doesn't loo
    """
    all_node_configs = [node_profs[node].enumerate_configs(max_replication_factor=3) for node in dag.nodes()]     
    all_pipeline_configs = itertools.product(*all_node_configs)
    num_valid_configs = 0
    best_config = None
    best_config_perf = None
    cur_index = 0
    for p_config in all_pipeline_configs:
        cur_index += 1
        if cur_index % 500 == 0:
            print("Processed {}".format(cur_index))
        cur_node_configs = {n.name: n for n in p_config}
        if not profiler.is_valid_pipeline_config(cur_node_configs):
            continue
        cur_config_perf = profiler.estimate_pipeline_performance_for_config(
            dag, scale_factors, cur_node_configs, node_profs)
        if cur_config_perf["latency"] <= latency_constraint and cur_config_perf["cost"] <= cost_constraint:
            if best_config is None:
                best_config = cur_node_configs
                best_config_perf = cur_config_perf
                print("Initializing config to {} ({})".format(best_config, best_config_perf))
            else:
                if cur_config_perf["throughput"] > best_config_perf["throughput"]:
                    best_config = cur_node_configs
                    best_config_perf = cur_config_perf
                    print("Updating config to {} ({})".format(best_config, best_config_perf))
        
    return best_config, best_config_perf

In [ ]:
start = datetime.now()
brute_force_optimizer(dag, scale_factors, node_profs, 7.0, 0.8)
end = datetime.now()
print("{}".format((end-start).total_seconds()))

In [ ]:
profiler.estimate_pipeline_performance_for_config(dag, scale_factors, node_configs, node_profs)

In [ ]:
groups = profs["alexnet"].groupby(["cloud","gpu_type","num_cpus_per_replica"])
groups

In [ ]:
for name, df in groups:
    print(name, df)